In [ ]:
import pandas as pd
import numpy as np

In [ ]:
path =  "../data/"

#dataset = "recsys17/"
dataset = "cb12/"

predict_path = path + dataset + "interim/predict/"
processed_path = path + dataset + "processed/"


algo_files = {
    "vae_int": "r_no_0.0_vae_test_14d_topks_60.csv",
    "vae_cont": "content_r_no_0.0_vae_test_14d_topks_60.csv",
    "sknn": "base/test_14d_cknn.csv",
    "ssknn": "base/test_14d_scknn.csv",
    "vsknn": "base/test_14d_vcknn.csv",
    "gru4rec": "base/test_14d_gru.csv",
    "bayes": "base/test_d14_bayes_test.csv",
    "iknn": "base/test_14d_iknn.csv",
    "bpr": "base/test_14d_bpr.csv",
    "pop": "base/test_14d_pop.csv",
    "prnn": "base/test_14d_prnn.csv"
}


In [ ]:
train_full_sessions = pd.read_csv(processed_path + "train_14d.csv", sep='\t')
test_sessions = pd.read_csv(processed_path + "test_14d.csv", sep='\t')

test_sessions['item_id'] = test_sessions['item_id'].apply(str)

train_item_len = len(train_full_sessions.item_id.unique())
test_item_len = len(test_sessions.item_id.unique())
merged_items = train_full_sessions.append(test_sessions, ignore_index=True)

merged_items['item_id'] = merged_items['item_id'].apply(str)


merged_item_len = len(merged_items.item_id.unique())

print("Training - Items: " + str(train_item_len))
print("Testing - Items: " + str(test_item_len))
print("Train + Test - Items: " + str(merged_item_len))

In [ ]:
def calc_coverage(all_preds):
    all_preds = set(np.concatenate(preds['predictions'].values))
    num_pred = len(all_preds)
    
    pred_item_total = len(all_preds.intersection(merged_items.item_id.unique()))
    total_ratio = pred_item_total/merged_item_len
    
    pred_item_test = len(all_preds.intersection(test_sessions.item_id.unique()))
    test_ratio = pred_item_test/test_item_len

    return num_pred, pred_item_total, total_ratio, pred_item_test, test_ratio

In [ ]:
for algo_name, algo_file in algo_files.items():
    preds = pd.read_csv(predict_path + algo_file, header=0, sep='\t', index_col=0)
    preds["predictions"] = preds["predictions"].str.split(",")
    all_preds = set(np.concatenate(preds['predictions'].values))
    res = calc_coverage(all_preds)
    print(f"{algo_name:8s}: total - test")
    print(f"Ratio   : {res[2]*100:.2f} - {res[4]*100:.2f}")
    print(f"#       : {res[1]} - {res[3]}")
    print()